# Predicting weekly fatality cases using SafeGraph mobility data

During the course of the pandemic, several prediction models for COVID-19 cases and fatalities have been developed. <a href="https://www.cdc.gov/coronavirus/2019-ncov/covid-data/forecasting-us.html">CDC publishes the projections</a> for some of these models, which in general use a variety of input data and epidemiological components (e.g., SEIR) and approaches (e.g., agent-based, curve fitting, statistical learning etc.). They are also making a variety of assumptions about the measures put in place to fight the pandemic. 

While initially I did not want to go down the rabbit hole of creating yet another model - and for the most part I succeeded in not doing this - I finally caved in and made an attempt to build a simple, short-term, prediction model for COVID-19 related fatalities. My idea is that these models do not seem (unless if I missed it, in which case I apologize -- or maybe I was just looking for an excuse to model) to incorporate actual mobility data. Virus transmission and consequently infections and deaths depend a lot on how people interact and move in space. <a href="https://www.safegraph.com/covid-19-data-consortium">SafeGraph</a> has provided a very detailed dataset on the mobility of people and the foot traffic of businesses during the course of the pandemic. It seemed natural to try and use some of the information in these data for projecting fatalities. The objective was to see whether simply mobility information is enough to predict (to a satisfactory extend) fatalities in short term (e.g., one or two weeks out). 

One of the information provided by the dataset is the numbe of devices tracked within a Census Block Group (CBG) that stayed completely at home during the day. We also know the numbe of devices from that specific CBG that are in the SafeGraph dataset, and hence we can estimate the fraction of devices $h_i(\tau)$ within CBG $i$, during day $\tau$ that stayed ompletely home. We can then get the average (weighted by the devices in each CBG) over the whole US ($h_{US}(\tau)$), which can then be aggregated weekly ($h_{US}(t)$, for week $t$). With $y(t)$ being the number of COVID-19-related deaths during week $t$ as well, we can build a prediction model for $y(t+1)$, using autoregressive features $[y(t-N),~y(t-N+1), \dots y(t-1)]$ and mobility features $[h_{US}(t-M)~,h_{US}(t-M+1),\dots,h_{US}(t-1)]$. Given the size of our dataset at this point (and the results from our <a href="https://arxiv.org/abs/2006.12720">Granger causality analysis</a>, we use $M=N=3$. 

We built a Long-Short Term Memory neural network with two layers, followed by a dropout and a dense layer. We also enable the dropout layer in the prediction phase which allows us to estimate the <a href="https://arxiv.org/pdf/1709.01907.pdf">uncertainty of the prediction in a Bayesian fashion</a>. We showcase the out-of-sample predictions for the last 4 weeks of data. For example, assuming $W$ weeks available, we predict the weekly fatality count for weeks $W-3,~W-2,~W-1,~W$, using a model trained only on data from weeks before. E.g., for predicting $y(W-3)$, we train the LSTM with data up to (and including) week $W-4$. For predicting $y(W-2)$, we train the LSTM with data up to (and including) week $W-3$ etc.

<B>To Do:</B> (i) Currently the prediction is for next week only. However, further out predictions are possible if you set the variable ```n_steps_out``` to the required value. I would be cautious though for trying to predict too far out since (a) there are not enough data points in the time-series, (b) it is not clear what will be the interventions put in place and how these will affect mobility etc. Nevertheless, there are ways to further model these. Additional variables can be used, but again my objective is to simply see whether just mobility and auto-regressive features can have good performance. Feel free to use the code and update with more additional features. 
(ii) Model hyperparameters (e.g., dropout rate, units/layer, number of hidden layers etc.) should be chosen through a validation step rather than selected a-priori. However, the training set size makes it challenging. Currently, I am using the 4 weeks between 6/9 and 7/6 for out-of-sample testing of the hand-picked hyperparameters, but I am planning on using them as validation set to choose the model hyperparameters (at some point). 
(iii) I will be <a href="https://docs.google.com/spreadsheets/d/10ymmdz3TXhGXLoH3l1uB7bb0N4wEGPvNvDXFF_Hoiaw/edit?usp=sharing">putting out my weekly predictions</a> with a couple of days delay from the start of the week since there is a lag in the SafeGraph data. However, I will not be using any information from these 3 first days of the week. 

In [1]:
import pandas as pd
import os 
import numpy as np
import pickle
import warnings
warnings.filterwarnings("ignore")

# data processing

def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
    return allFiles

url = 'https://raw.githubusercontent.com/nytimes/covid-19-data/master/us.csv'
us_data = pd.read_csv(url)

# mobility data -- you should change this to your directory of SafeGraph data
directory_sd = "/Volumes/Seagate Desktop Media/SafeGraph/social-distancing/v2/2020/"

files = getListOfFiles(directory_sd)

# this pickle file contains the fraction of people staying completely home every day that we have already processed
stay_home_pcg = pickle.load(open("mobility_tsUSAfr.p","rb"))
stay_home_pcg = {key:val for key, val in stay_home_pcg.items() if key > '2020-01-20'}


# only new data are processed and added to stay_home_pcg dictionary

for f in files:
        print(f)
        if '.DS_Store' in f:
                continue
        date = f.split("/")[-1].split("-social")[0]
        if (date < '2020-01-21') or (date in stay_home_pcg):
            continue
        tmp = pd.read_csv(f,compression="gzip")
        # fraction of people staying completely at home at each CBG
        tmp['home_pcg'] = tmp.completely_home_device_count/tmp.device_count
        try:
            stay_home_pcg[date] = np.average(tmp['home_pcg'], weights=tmp['device_count'])
        except:
            print(tmp.head())
            
pickle.dump( stay_home_pcg, open( "mobility_tsUSAfr.p", "wb" ) )
                

/Volumes/Seagate Desktop Media/SafeGraph/social-distancing/v2/2020/.DS_Store
/Volumes/Seagate Desktop Media/SafeGraph/social-distancing/v2/2020/01/01/2020-01-01-social-distancing.csv.gz
/Volumes/Seagate Desktop Media/SafeGraph/social-distancing/v2/2020/01/02/2020-01-02-social-distancing.csv.gz
/Volumes/Seagate Desktop Media/SafeGraph/social-distancing/v2/2020/01/03/2020-01-03-social-distancing.csv.gz
/Volumes/Seagate Desktop Media/SafeGraph/social-distancing/v2/2020/01/04/2020-01-04-social-distancing.csv.gz
/Volumes/Seagate Desktop Media/SafeGraph/social-distancing/v2/2020/01/05/2020-01-05-social-distancing.csv.gz
/Volumes/Seagate Desktop Media/SafeGraph/social-distancing/v2/2020/01/06/2020-01-06-social-distancing.csv.gz
/Volumes/Seagate Desktop Media/SafeGraph/social-distancing/v2/2020/01/07/2020-01-07-social-distancing.csv.gz
/Volumes/Seagate Desktop Media/SafeGraph/social-distancing/v2/2020/01/08/2020-01-08-social-distancing.csv.gz
/Volumes/Seagate Desktop Media/SafeGraph/social-dis

In [2]:
stay_home_pcg = pickle.load(open("mobility_tsUSAfr.p","rb"))
stay_home_pcg = {key:val for key, val in stay_home_pcg.items() if key > '2020-01-20'}

mobility = []

#stay_home_pcg = {key:val for key, val in stay_home_pcg.items() if key < '2020-07-04'}

for i in range(len(stay_home_pcg)):
    try:
        mobility.append(stay_home_pcg[us_data.iloc[i]['date']])
    except:
        pass

granger_data= us_data[us_data['date'] <= max(stay_home_pcg.keys())]
granger_data['mobility'] = mobility

diff_deaths = [0]+[granger_data.iloc[i]['deaths']-granger_data.iloc[i-1]['deaths'] for i in range(1,len(granger_data))]
granger_data['daily_deaths'] = diff_deaths
diff_cases = [0]+[granger_data.iloc[i]['cases']-granger_data.iloc[i-1]['cases'] for i in range(1,len(granger_data))]
granger_data['daily_cases'] = diff_cases

# find weekly numbers/indices for less noise

weekly_deaths = []
weekly_mobility = []
week_dates = []

d = 0
while d+7 <= len(granger_data):
    weekly_deaths.append(sum(granger_data.iloc[d:(d+7)]['daily_deaths']))
    weekly_mobility.append(np.mean(granger_data.iloc[d:(d+7)]['mobility']))
    week_dates.append(granger_data.iloc[d]['date'])
    d = d+7
    
    
weekly_granger = pd.DataFrame(list(zip(week_dates,weekly_deaths, weekly_mobility)), columns =['date','deaths', 'mobility']) 

In [3]:
## build LSTM model
import keras
import keras.backend as K
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers import Dropout
from numpy import array
from numpy import hstack

# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps_in
		out_end_ix = end_ix + n_steps_out-1
		# check if we are beyond the dataset
		if out_end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

class Dropout(keras.layers.Dropout):

    def __init__(self, rate, training=None, noise_shape=None, seed=None, **kwargs):
        super(Dropout, self).__init__(rate, noise_shape=None, seed=None,**kwargs)
        self.training = training

        
    def call(self, inputs, training=None):
        if 0. < self.rate < 1.:
            noise_shape = self._get_noise_shape(inputs)

            def dropped_inputs():
                return K.dropout(inputs, self.rate, noise_shape,
                                 seed=self.seed)
            if not training: 
                return K.in_train_phase(dropped_inputs, inputs, training=self.training)
            return K.in_train_phase(dropped_inputs, inputs, training=training)
        return inputs

TESTING = False
    
if TESTING:
    tin_seq1 = array(list(weekly_granger['mobility']))
    tin_seq2 = array(list(weekly_granger['deaths']))


    #out of sample predictions for 4 last weeks 
    for w in list(range(len(weekly_granger)-4,len(weekly_granger))):
        in_seq1 = array([[weekly_granger.iloc[i]['mobility']] for i in range(w)])
        in_seq2 = array([[weekly_granger.iloc[i]['deaths']] for i in range(w)])
        out_seq = array([in_seq2[i] for i in range(1,len(in_seq2))] + [0])[0:w]
        # horizontally stack columns
        in_seq1 = in_seq1.reshape((len(in_seq1), 1))
        in_seq2 = in_seq2.reshape((len(in_seq2), 1))
        out_seq = out_seq.reshape((len(out_seq), 1))
        dataset = hstack((in_seq1, in_seq2, out_seq))
        # choose a number of time steps
        n_steps_in, n_steps_out = 3, 1
        # convert into input/output
        X, y = split_sequences(dataset, n_steps_in, n_steps_out)
        n_features = X.shape[2]
        model = Sequential()
        model.add(LSTM(50, activation='relu', return_sequences = True, input_shape=(n_steps_in, n_features)))
        model.add(LSTM(50,activation='relu'))
        model.add(Dropout(rate=0.15, training=True))
        model.add(Dense(n_steps_out))
        model.compile(optimizer='adam', loss='mse')
        model.fit(X, y, epochs=100, verbose=0)
        # predict
        x_input = array([[tin_seq1[w-3], tin_seq2[w-3]], [tin_seq1[w-2], tin_seq2[w-2]], [tin_seq1[w-1], tin_seq2[w-1]]])
        x_input = x_input.reshape((1, n_steps_in, n_features))
        yhat = [model.predict(x_input,verbose=0)[0,0] for _ in range(1000)]
        print("Prediction for week",w)
        print("Average: ", np.mean(yhat))
        print("10-90th percentile: [",np.quantile(yhat,q=0.1),",",np.quantile(yhat,q=0.9),"]")
        print("True:", tin_seq2[w])
        print("============================================")
    

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Prediction for week 20
Average:  6114.9033
10-90th percentile: [ 4225.169677734375 , 7631.170263671876 ]
True: 5072
Prediction for week 21
Average:  5620.69
10-90th percentile: [ 4056.50107421875 , 7008.841943359375 ]
True: 4118
Prediction for week 22
Average:  5454.167
10-90th percentile: [ 2951.2032470703125 , 8078.471240234376 ]
True: 5827
Prediction for week 23
Average:  4120.576
10-90th percentile: [ 3267.0211914062497 , 4837.9677734375 ]
True: 4171


In [5]:
# predict for next week

# features from last three weeks 
w = len(weekly_granger)
in_seq1 = array([[weekly_granger.iloc[i]['mobility']] for i in range(w)])
in_seq2 = array([[weekly_granger.iloc[i]['deaths']] for i in range(w)])
out_seq = array([in_seq2[i] for i in range(1,len(in_seq2))] + [0])[0:w]
# horizontally stack columns
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
dataset = hstack((in_seq1, in_seq2, out_seq))
# choose a number of time steps
n_steps_in, n_steps_out = 3, 1
# convert into input/output
X, y = split_sequences(dataset, n_steps_in, n_steps_out)
n_features = X.shape[2]
model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences = True, input_shape=(n_steps_in, n_features)))
model.add(LSTM(50,activation='relu'))
model.add(Dropout(rate=0.15, training=True))
model.add(Dense(n_steps_out))
model.compile(optimizer='adam', loss='mse')
model.fit(X, y, epochs=100, verbose=0)
x_input = array([[tin_seq1[w-3], tin_seq2[w-3]], [tin_seq1[w-2], tin_seq2[w-2]], [tin_seq1[w-1], tin_seq2[w-1]]])
x_input = x_input.reshape((1, n_steps_in, n_features))
yhat = [model.predict(x_input,verbose=0)[0,0] for _ in range(1000)]
print("Prediction for week",w)
print("Average: ", np.mean(yhat))
print("10-90th percentile: [",np.quantile(yhat,q=0.1),",",np.quantile(yhat,q=0.9),"]")
print("============================================")

Prediction for week 24
Average:  4185.3423
10-90th percentile: [ 3480.8141357421873 , 4869.049902343751 ]
